# Black-Scholes Model
The Black-Scholes model is a mathematical model for pricing options. This model is particularly known for the Black-Scholes formula, which provides a theoretical estimate of the price of European-style options.

## Key Concepts

1. **European Options:** These options can only be exercised at expiration, unlike American options, which can be exercised at any time before expiration.
2. **Call Option:** Gives the holder the right (but not the obligation) to buy an asset at a specified price (strike price) on the expiration date.
3. **Put Option:** Gives the holder the right (but not the obligation) to sell an asset at a specified price on the expiration date.

## The Black-Scholes Formula

The Black-Scholes formula calculates the price of European call and put options. The key inputs to the model are:

- $S$: Current price of the stock
- $K$: Strike (or exercise) price of the option
- $T$: Time to expiration (in years)
- $r$: Risk-free interest rate (annualized)
- $σ$: Volatility of the stock (annualized) - Standard deviation of returns
- $N(\cdot)$: Cumulative distribution function of the standard normal distribution


The formulas for the price of a European call option $C$ and put option $P$ are:

<br>

$$C = S N(d_1) - K e^{-rT} N(d_2) $$

<br>

$$ P = K e^{-rT} N(-d_2) - S N(-d_1) $$

<br>

where

$$ d_1 = \frac{\ln(S/K) + (r + \sigma^2 / 2)T}{\sigma \sqrt{T}} $$

<br>

$$ d_2 = d_1 - \sigma \sqrt{T} $$

## Assumptions of the Black-Scholes Model

1. The stock price follows a geometric Brownian motion with constant drift and volatility.
2. There are no dividends paid out during the life of the option.
3. The markets are efficient (i.e., market movements cannot be predicted).
4. No transaction costs or taxes.
5. The risk-free rate and volatility of the stock are constant over the option's life.
6. The option can only be exercised at expiration (European option).

In [ ]:
import numpy as np
from scipy.stats import norm

In [ ]:
def calc_d1(S, K, T, r, sigma):
    return (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))

def calc_d2(d1, sigma, T):
    return d1 - sigma * np.sqrt(T)

def black_scholes(S, K, T, r, sigma, opt_type):
    assert opt_type in ["c", "p"], "Invalid option type. Choose 'c' for 'call' or 'p' for 'put'."

    d1 = calc_d1(S, K, T, r, sigma)
    d2 = calc_d2(d1, sigma, T)

    # Calculate the price of the option
    if opt_type == "c":
        price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    else:
        price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)

    return price

# Black-Scholes Greeks

The Black-Scholes model incorporates several key sensitivities known as the "Greeks." These Greeks measure the sensitivity of the option's price to various factors.

## Delta (Δ)

Delta measures the **rate of change of the option’s price with respect to the underlying asset**.

<br>

$$ \Delta = \frac{∂ V}{∂ S} $$

<br>

- **Call Option**: Delta ranges from 0 to 1. A delta of 0.5 means that for every 1 unit increase in the stock price, the call option’s price will increase by 0.50.
- **Put Option**: Delta ranges from -1 to 0. A delta of -0.5 means that for every 1 unit increase in the stock price, the put option’s price will decrease by 0.50.

<br>

### Formula:
$$ \Delta_{call} = N(d_{1}) $$
$$ \Delta_{put} = -N(-d_{1}) = N(d_{1}) - 1 $$

In [ ]:
def calc_delta(S, K, T, r, sigma, opt_type):
    d1 = calc_d1(S, K, T, r, sigma)
    if opt_type == "c":
        delta = norm.cdf(d1)
    else:
        delta = norm.cdf(d1) - 1

    return delta

## Gamma (Γ)

Gamma measures the **rate of change of delta with respect to changes in the underlying price**.

<br>

$$ \Gamma = \frac{∂ Δ}{∂ S} = \frac{∂^2 V}{∂ S^2} $$

<br>

It shows how much the delta will change as the underlying asset price changes. Gamma is highest when the option is at-the-money and decreases as the option moves further in-the-money or out-of-the-money.

<br>

### Formula:
$$ \Gamma = \frac{N'(d1)}{S\sigma\sqrt{T}} $$

<br>

where $N'(x)$ is the probability density function of the standard normal distribution

In [ ]:
def calc_gamma(S, K, T, r, sigma):
    d1 = calc_d1(S, K, T, r, sigma)
    gamma = norm.pdf(d1, 0, 1) / (S * sigma * np.sqrt(T))

    return gamma

## Vega (ν)

Vega measures the **rate of change of the option’s price with respect to the volatility of the underlying asset**. Vega is the same for both call and put options and is higher for at-the-money options.

<br>

$$ \nu = \frac{∂V}{∂σ} $$

<br>

### Formula:
$$ \nu = S \sqrt{T} N'(d_{1}) $$

In [ ]:
def calc_vega(S, K, T, r, sigma):
    d1 = calc_d1(S, K, T, r, sigma)
    vega = S * np.sqrt(T) * norm.pdf(d1, 0, 1)

    vega = vega / 100 # transform the measure to 1% change in volatility

    return vega

## Theta (Θ)

Theta measures the **rate of change of the option’s price with respect to time**, also known as time decay. It represents the amount by which the option’s price decreases as time to expiration decreases.

- **Call and Put Options**: Theta is usually negative, indicating that the option loses value as time passes.

$$ Θ = -\frac{∂V}{∂T}$$

<br>

### Formula:
$$ \Theta_{call} = -\frac{S N'(d_{1}) \sigma}{2\sqrt{T}} - r K e^{-rT} N(d_{2}) $$
$$ \Theta_{put} = -\frac{S N'(d_{1}) \sigma}{2\sqrt{T}} + r K e^{-rT} N(-d_{2}) $$

In [ ]:
def calc_theta(S, K, T, r, sigma, opt_type):
    d1 = calc_d1(S, K, T, r, sigma)
    d2 = calc_d2(d1, sigma, T)

    if opt_type == "c":
        theta = -S * norm.pdf(d1, 0, 1) * sigma / (2 * np.sqrt(T)) - r * K * np.exp(-r * T) * norm.cdf(d2)
    else:
        theta = -S * norm.pdf(d1, 0, 1) * sigma / (2 * np.sqrt(T)) + r * K * np.exp(-r * T) * norm.cdf(-d2)

    theta = theta / 365 # convert to days

    return theta

## Rho (ρ)

Rho measures the ** rate of change of the option’s price with respect to the risk-free interest rate**.

<br>

$$ ρ = \frac{∂V}{∂r}$$

<br>

- **Call Option**: Rho is positive, meaning that an increase in interest rates will increase the price of a call option.
- **Put Option**: Rho is negative, meaning that an increase in interest rates will decrease the price of a put option.

<br>

### Formula:
$$ \rho_{call} = K T e^{-rT} N(d_{2}) $$
$$ \rho_{put} = -K T e^{-rT} N(-d_{2}) $$

In [ ]:
def calc_rho(S, K, T, r, sigma, opt_type):
    d2 = calc_d2(calc_d1(S, K, T, r, sigma), sigma, T)

    if opt_type == "c":
        rho = K * T * np.exp(-r * T) * norm.cdf(d2)
    else:
        rho = -K * T * np.exp(-r * T) * norm.cdf(-d2)

    rho = rho / 100 # transform the measure to 1% change in interest rate

    return rho

# Example & Sanity Check

In [ ]:
!pip install py_vollib

In [ ]:
from py_vollib.black_scholes import black_scholes as bs
import py_vollib.black_scholes.greeks.analytical as greeks

/usr/local/lib/python3.10/dist-packages/py_lets_be_rational/numba_helper.py:10: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  return jit(*jit_args, **jit_kwargs)(fun)


In [ ]:
# Example parameters
S = 100  # Current stock price
K = 100  # Strike price
T = 1    # Time to expiration in years
r = 0.05  # Risk-free interest rate
sigma = 0.2  # Volatility
opt_type = "c"

In [ ]:
print("Option Price: ", [round(x,3) for x in [black_scholes(S, K, T, r, sigma, opt_type), bs("c", S, K, T, r, sigma)]])
print("Delta: ", [round(x,3) for x in [calc_delta(S, K, T, r, sigma, opt_type), greeks.delta(opt_type, S, K, T, r, sigma)]])
print("Gamma: ", [round(x,3) for x in [calc_gamma(S, K, T, r, sigma), greeks.gamma(opt_type, S, K, T, r, sigma)]])
print("Vega: ", [round(x,3) for x in [calc_vega(S, K, T, r, sigma), greeks.vega(opt_type, S, K, T, r, sigma)]])
print("Theta: ", [round(x,3) for x in [calc_theta(S, K, T, r, sigma, opt_type), greeks.theta(opt_type, S, K, T, r, sigma)]])
print("Rho: ", [round(x,3) for x in [calc_rho(S, K, T, r, sigma, opt_type), greeks.rho(opt_type, S, K, T, r, sigma)]])

Option Price:  [10.451, 10.451]
Delta:  [0.637, 0.637]
Gamma:  [0.019, 0.019]
Vega:  [0.375, 0.375]
Theta:  [-0.018, -0.018]
Rho:  [0.532, 0.532]


# Side note: Cumulative Distribution Function (CDF)
The CDF is a fundamental concept in probability theory and statistics that describes the probability that a random variable takes on a value less than or equal to a specified value. For a random variable $X$, the CDF, denoted as $F(x)$, is defined as:

$$ F(x) = P(X \leq x) $$

where $P$ denotes the probability.

## Properties of the CDF

1. **Non-decreasing**: The CDF is a non-decreasing function. This means $F(x_1) \leq F(x_2) $ for $ x_1 < x_2 $.
2. **Limits**:
   - As $ x $ approaches negative infinity, $ F(x) $ approaches 0.
   - As $ x $ approaches positive infinity, $ F(x) $ approaches 1.
3. **Right-continuous**: The CDF is right-continuous, meaning there are no jumps when approaching from the right.


## CDF for Discrete Random Variables

For a discrete random variable $ X $ with possible values $ x_1, x_2, \ldots $ and corresponding probabilities $ p_1, p_2, \ldots $:

<br>

$$ F(x) = P(X \leq x) = \sum_{x_i \leq x} P(X = x_i) = \sum_{x_i \leq x} p_i $$

<br>

For example: Consider a discrete random variable $ X $ that can take on values 1, 2, and 3 with probabilities $ P(X=1) = 0.2 $, $ P(X=2) = 0.5 $, and $ P(X=3) = 0.3 $. The CDF $ F(x) $ is:

<br>

- For $ x < 1 $, $ F(x) = 0 $
- For $ 1 \leq x < 2 $, $ F(x) = 0.2 $
- For $ 2 \leq x < 3 $, $ F(x) = 0.7 $
- For $ x \geq 3 $, $ F(x) = 1.0 $


## CDF for Continuous Random Variables

For a continuous random variable $ X $ with probability density function (PDF) $ f(x) $:

<br>

$$ F(x) = P(X \leq x) = \int_{-\infty}^x f(t) \, dt $$

<br>
For example: Consider a continuous random variable $ X $ with a PDF $ f(x) = e^{-x} $ for $ x \geq 0 $ (an exponential distribution with parameter $ \lambda = 1 $). The CDF $ F(x) $ is:

$$ F(x) = \int_{-\infty}^x e^{-t} \, dt = \left\{
\begin{array}{ll}
0 & \text{for } x < 0 \\
1 - e^{-x} & \text{for } x \geq 0
\end{array}
\right. $$